In [21]:
import scrapy
import requests
import csv
import time
import random

verde = '\033[92m'
amarelo = '\033[93m'
vermelho = '\033[91m'
azul = '\033[94m'
fim = '\033[0m'

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/128.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.5 Safari/605.1.15',
]

dados_coletados_geral = []

print(f'{azul}\n------------------Iniciado Imobiliária Maximize------------------{fim}')
contador_imoveis = 0
pagina = 1



def getattr(link,headers):
    
    html_content = requests.get(link, headers = headers, timeout=20).text
    response = scrapy.http.HtmlResponse(url = link, body = html_content, encoding = "utf-8")
    
    preco = response.xpath(
    '//span[contains(text(), "Locação")]/following-sibling::p[1]/text()'
    ).get()
    if not preco:
        preco = response.xpath(
        '//div[contains(text(), "Locação")]/following-sibling::p[1]/text()'
        ).get() 


    quartos = response.xpath(
        '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Quartos")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
    ).get()

    suites = response.xpath(
        '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Suíte")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
    ).get()

    area = response.xpath(
        '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Área Total")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
    ).get()

    if not area:
        area = response.xpath(
            '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Área Privativa")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
        ).get()
    if not area:
        area = response.xpath(
            '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Área Construída")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
        ).get()

    if not area:
        area = response.xpath(
            '//div[contains(@class,"big-card-property-facility-exebicao")][div[contains(text(),"Área Terreno")]]//div[contains(@class,"text-r-bold-exebicao")]/text()'
        ).get()
    if not area:
        descriacao = response.xpath(
        '//*[@id="w-node-_501b00e3-5969-2d2f-4246-33f42ad8c0cc-51cf1fd3"]/div[2]/div[2]/p'
        ).get()
        descriacao = descriacao.split('<br>')
        for part in descriacao:
            if 'm²' in part:
                # print(part)
                area = part.split(':')[-1].replace('.','').split(' ')
                for i,a in enumerate(area):
                    if 'm²' in a:
                        # area = a
                        if a == 'm²':
                            try:
                                area = area[i-1]+a
                            except:
                                area = ''
                        else:
                            area = a
                break
        # print(descriacao)
    condominio = response.xpath(
        '//span[contains(text(), "Valor de Condominio:")]/following-sibling::p[1]/text()'
        ).get()



    return (preco,condominio,quartos,suites,area)

for pagina in range(1,6):
    print(f"{azul}Coletando página: {pagina}{fim}")
    url_geral = f'https://www.maximizeimobiliaria.com.br/filtro2/list/locacao/todos/todas/todos/0-10000000/todos/{pagina}/ASC'
    random_user_agent = random.choice(user_agents)
    headers = {"User-Agent": random_user_agent}

    try:
        html_content = requests.get(url_geral, headers = headers, timeout=20).text
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
        print(f"{amarelo}Erro de conexão/timeout na página {pagina}. Tentando novamente em 7 segundos...{fim}")
        time.sleep(7)
        continue

    # time.sleep(random.uniform(2, 5))
    response = scrapy.http.HtmlResponse(url = url_geral, body = html_content, encoding = "utf-8")

    imoveis_na_pagina = response.css("div.card-property-exebicao.raw-exebicao.w-dyn-item")
    if len(imoveis_na_pagina) == 0:
        break

    for imovel in imoveis_na_pagina:
        contador_imoveis += 1

        titulo = imovel.css("h5.heading-2-exebicao::text").get()

        tipo = None
        if "Apartamento" or "Casa" or "Sobrado" in titulo:
            tipo = "Residencial"
        elif "Barracão" in titulo:
            tipo = "Barracão" 
        elif "Sala" in titulo:
            tipo = "Sala Comercial"
        else:
            tipo = "Não identificado"

        link = imovel.css("div.slide-exebicao.w-slide>a::attr(href)").get()

        preco,condominio,quartos,suites,area = getattr(link,headers)

        dados_coletados_geral.append({
            "Titulo do imóvel": titulo,
            "Tipo do imóvel": tipo,
            "Preço": preco,
            "Valor Condomínio": condominio,
			"Bairro": imovel.css("div.card-property-description-exebicao>div::text").get(),
    		"Link para o anúncio": link,
            "Origem": 'Maximize Imobiliária',
            "Quartos": quartos,
            "Suítes": suites,
            "Área do imóvel": area
        })
        print(f"{contador_imoveis}.{verde} Coletado:", titulo, f'{fim}')
        # time.sleep(0.5)
    # time.sleep(1)
    # pagina += 1
    # break

print(f'\n{azul}------------------Finalizado Imobiliária Maximize------------------{fim}')

with open("dados_scraper.csv", "w", newline = "", encoding = "utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Titulo do imóvel", "Tipo", "Preço", 'Valor Condominio', "Quartos", "Suítes", 'Área Total', "Bairro", "Link para o anúncio", 'Origem'])
    for dado in dados_coletados_geral:
        writer.writerow([dado["Titulo do imóvel"], dado["Tipo do imóvel"], dado["Preço"], dado["Valor Condomínio"], dado["Quartos"], dado["Suítes"], dado["Área do imóvel"], dado["Bairro"], dado["Link para o anúncio"], dado['Origem']])
    print(f"{verde}Salvo em CSV!{fim}")



------------------Iniciado Imobiliária Maximize------------------
Coletando página: 1
1. Coletado: Sobrado à venda, Centro, TOLEDO - PR 
2. Coletado: Barracão, 338 m² - venda por R$ 1.350.000,00 ou aluguel por R$ 6.000,00/mês - Ja 
3. Coletado: Sobrado para venda e locação, no Jardim Pancera, com uma suíte e mais dois quartos, piscina, 
4. Coletado: Casa para locação e venda com 03 quartos e piscina, Vila Becker, TOLEDO - PR 
5. Coletado: Apartamento semi-mobiliado  à venda e locação no  Centro de Toledo 
6. Coletado: Apartamento à venda, Jardim La Salle, TOLEDO - PR 
7. Coletado: Apartamento à venda, Jardim Gisela, TOLEDO - PR 
8. Coletado: Sala, 26 m² - venda por R$ 500.000,00 ou aluguel por R$ 2.500,00/mês - Centro -  
9. Coletado: Sala Comercial para locação, Centro, TOLEDO - PR 
10. Coletado: Sala Comercial para locação, Centro, TOLEDO - PR 
11. Coletado: Sala Comercial para locação no Centro de Toledo - PR 
12. Coletado: SALA COMERCIAL COM 220,00 M  - 01 VAGA DE GARAGEM - COPA -

# Regressao Linear ScL